# Media

(Last updated: May 30, 2024)

Since around 2020, I have been keeping track of almost all media I consume - mostly books and games, but also concerts (but not CD's), movies, museums, and other forms of culture. Here, you can find a quick summary of what I've been up to

## Currently enjoying

In [232]:
import pandas as pd
df = pd.read_csv("media.csv")

verb_dict = {"Book":"read","Game":"played","Movie":"watched","Music":"visited","Museum":"visited"}
print(f"Last {verb_dict[df['type'].iloc[-1]]}: {df['name'].iloc[-1]} by {df['creator'].iloc[-1]}")
score_dict = {10:'close to perfect!', 9:'fantastic!', 8:'great', 7:'good', 6:'okay', 5:'meh...', 4:'not great', 3:'bad', 2:'terrible', 1: 'abhorrent', 0:'the worst!'}
print(f"Which I thought was: {score_dict[round(df['score'].iloc[-1])]}")

Last played: Elden Ring by FromSoftware
Which I thought was: close to perfect!


## All-time favorites

In [233]:
df3 = df.copy()
sample = df3[df3['score'] > 9.5].sample(3).reset_index()
print("Some works I can wholeheartedly recommend are:")
for i in range(len(sample)):
    print(f"* {sample['name'][i]} by {sample['creator'][i]}, which I rated at {sample['score'][i]}")

Some works I can wholeheartedly recommend are:
* Disco Elysium by ZA/UM, which I rated at 9.8
* Roshambo, of Waarom we Steen-Papier-Schaar Spelen by DraeCKI, which I rated at 9.9
* Elden Ring by FromSoftware, which I rated at 9.9


## What I play

In [267]:
import plotly.express as px
def rounder(score):
    score = float(round(score*2))/2
    if score == 10:
        return "score≈" + str(9.9)
    return "score≈" + str(score)

df4 = df.copy()
df4 = df4.groupby(['type', 'score']).size().rename_axis(['type', 'score']).reset_index(name='counts')
df4['scoreactual'] = df4['score'].apply(lambda x: rounder(x))
df4 = df4.sort_values(by=['type', 'score'])
fig = px.sunburst(df4, path=['type', 'scoreactual'], values='counts', labels=[''],
                    hover_data={
                    })
fig.update_layout(margin={'t':5,'l':5,'b':5,'r':5}, showlegend=False)
fig.update_traces(sort=False, hovertemplate='<b>Amount: %{value} </b>')
fig.show()

## How much I play

In [257]:
import plotly.express as px
df2 = df.copy()
df2 = df2.groupby(['type', 'date']).size().rename_axis(['type', 'date']).reset_index(name='counts')
df2[['year','month']] = df2['date'].str.split('-',expand=True)
df2 = df2.drop(columns=['date'])
types = []; counts = []; years = []; months = []
for year in range(min(df2['year'].astype(int)), max(df2['year'].astype(int)) + 1):
    for month in range(1, 13):
        actualmonth = '0' * (2 - len(str(month))) + str(month)
        for type in df2['type'].unique():
            if not ((df2['year'] == str(year)) & (df2['month'] == actualmonth) & (df2['type'] == type)).any():
                types.append(type); counts.append("0"); years.append(str(year)); months.append(actualmonth)
for year in range(min(df2['year'].astype(int)), max(df2['year'].astype(int)) + 1):
    for month in range(1, 13):
        actualmonth = '0' * (2 - len(str(month))) + str(month)
        types.append("All"); counts.append(df2[(df2['year'] == str(year)) & (df2['month'] == actualmonth)]['counts'].sum()); years.append(str(year)); months.append(actualmonth)
added_rows = pd.DataFrame({'type':types, 'counts':counts, 'year':years, 'month':months})
df2 = pd.concat([df2, added_rows], ignore_index=True)
df2 = df2.sort_values(by=['year', 'month'])
fig = px.line(df2, x='month', y='counts', color='year', animation_frame='type',
              labels={
                     "month": "Month",
                     "counts": "Amount",
                     "year": "Year"
                 },
                 hover_data={
                            'type':False,
                            'month':False,
                            'counts':False
                        })
fig.update_layout(margin={'t':10,'l':0,'b':0,'r':0}, yaxis_range=[0,10], xaxis_range=[0,11],
                  xaxis = dict(
                    tickmode = 'array',
                    tickvals = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
                    ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
                ), showlegend=False)
fig['layout']['sliders'][0]['pad']=dict(r= 10, t= 20, b=20)
fig["layout"].pop("updatemenus")
fig.show()
